# Creating images using shapes and simple simulation for PET
This exercise shows how to create images via geometric shapes. It then uses forward projection without
and with attenuation. Exercises are given for extending the simulation to include noise and other parts of the PET model.

Please note that the functionality for geometric shapes is currently specific to `sirf.STIR` as it relies on STIR classes.

It is recommended you complete the [Introductory](../Introductory) notebooks first (or alternatively the [display_and_projection.ipynb](display_and_projection.ipynb)). There is some overlap with [acquisition_model_mr_pet_ct.ipynb](../Introductory/acquisition_model_mr_pet_ct.ipynb), but here we use some geometric shapes to create an image, add attenuation and go into more detail about PET specifics.

Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: 8th of September 2016  
Second version: 17th of May 2018  
Third version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2018, 2021 University College London.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'image_creation_and_simulation')

In [ ]:
import notebook_setup

#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

In [ ]:
#%% Use the 'pet' prefix for all SIRF functions
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet
from sirf.Utilities import show_2D_array, show_3D_array, examples_data_path
from sirf_exercises import exercises_data_path

In [ ]:
# define the directory with input files
data_path = os.path.join(examples_data_path('PET'), 'brain')

# Creation of images

In [ ]:
#%% Read in image
# We will use an image provided with the demo to have correct voxel-sizes etc
image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
print(image.dimensions())
print(image.voxel_sizes())

In [ ]:
#%% create a shape
shape = pet.EllipticCylinder()
# define its size (in mm)
shape.set_length(50)
shape.set_radii((40, 30))
# centre of shape in (x,y,z) coordinates where (0,0,0) is centre of first plane
shape.set_origin((20, -30, 60))

In [ ]:
#%% add the shape to the image
# first set the image values to 0
image.fill(0)
image.add_shape(shape, scale=1)

In [ ]:
#%% add same shape at different location and with different intensity
shape.set_origin((40, -30, -60))
image.add_shape(shape, scale=0.75)

In [ ]:
#%% show the phantom image as a sequence of transverse images
show_3D_array(image.as_array());

# Simple simulation
Let's first do simple ray-tracing without attenuation

In [ ]:
#%% Create a SIRF acquisition model
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
# Specify sinogram dimensions via the template
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
# Now set-up our acquisition model with all information that it needs about the data and image.
acq_model.set_up(template,image); 

In [ ]:
#%% forward project this image and display all sinograms
acquired_data_no_attn = acq_model.forward(image)
acquired_data_no_attn_array = acquired_data_no_attn.as_array()[0,:,:,:]
show_3D_array(acquired_data_no_attn_array);

In [ ]:
#%% Show every 8th view 
# Doing this here with a complicated one-liner...
show_3D_array(
    acquired_data_no_attn_array[:,0:acquired_data_no_attn_array.shape[1]:8,:].transpose(1,0,2),
    show=False)
# You could now of course try the animation of the previous demo...

# Adding attenuation
Attenuation in PET follows the Lambert-Beer law:

$$\exp\left\{-\int\mu(x) dx\right\},$$

with $\mu(x)$ the linear attenuation coefficients (roughly proportional to density), 
and the line integral being performed between the 2 detectors.

It is a specific "feature" of PET that the attenuation factor only depends on the line integral between the detectors, and not on the voxel location (this is different in SPECT). Therefore, it is common practice to model PET acquisitions by first doing ray-tracing, followed by multiplication by attenuation factors (which have the "shape" of acquired data).

In SIRF, we do this by including an `AcquisitionSensitivityModel` object in the `AcquisitionModel`. The rationale for the name is that attenuation reduces the sensitivity of the detector-pair. Let's do this now.

## Create an attenuation image
We will use the "emission" image as a template for sizes. This is easiest as the underlying STIR projector has a limitation that the voxel-size in the axial dimension has to be related to the ring spacing of the scanner. (Note that the voxel-sizes in the 2 other directions can be arbitrary).

In [ ]:
attn_image = image.get_uniform_copy(0)
#%% create a shape for a uniform cylinder in the centre
shape = pet.EllipticCylinder()
shape.set_length(150)
shape.set_radii((60, 60))
shape.set_origin((0, 0, 40))
# add it to the attenuation image with mu=-.096 cm^-1 (i.e. water)
attn_image.add_shape(shape, scale=0.096)

In [ ]:
#%% show the phantom image as a sequence of transverse images
show_3D_array(attn_image.as_array());

In [ ]:
#%% Let's do a quick check that the values are fine
attn_image.max()

## Create the acquisition sensitivity model
The variable will be called `asm_attn` to indicate that it's the "acquisition sensitivity model for attenuation".

In [ ]:
# First create the ray-tracer
acq_model_for_attn = pet.AcquisitionModelUsingRayTracingMatrix()
# Now create the attenuation model
asm_attn = pet.AcquisitionSensitivityModel(attn_image, acq_model_for_attn)

In [ ]:
# Use this to find the 'detection efficiencies' as sinograms
asm_attn.set_up(template)
attn_factors = asm_attn.forward(template.get_uniform_copy(1))
# We will store these directly as an `AcquisitionSensitivityModel`, 
# such that we don't have to redo the line integrals
asm_attn = pet.AcquisitionSensitivityModel(attn_factors)

In [ ]:
#%% check a single sinogram (they are all the same for this simple case)
show_2D_array('Attenuation factor sinogram', attn_factors.as_array()[0,5,:,:]);

In [ ]:
#%% check a profile
# we'll use TOF bin=0, sinogram=5, view=0
plt.figure()
plt.plot(attn_factors.as_array()[0,5,0,:]);

## Create a SIRF acquisition model incorporating attenuation
We now add the attenuation to the full acquisition model.

In [ ]:
# start with ray-tracing
acq_model_with_attn = pet.AcquisitionModelUsingRayTracingMatrix()
# add the 'sensitivity'
acq_model_with_attn.set_acquisition_sensitivity(asm_attn)
# set-up
acq_model_with_attn.set_up(template,image);

All done!

## Use the new acquisition model
Using `acq_model_with_attn` works exactly the same as without attenuation. Overhead in computation is also quite small as the attenuation factors were computed above.

In [ ]:
#%% forward project the original image, now including attenuation modelling, and display all sinograms
acquired_data_with_attn = acq_model_with_attn.forward(image)
acquired_data_with_attn_array = acquired_data_with_attn.as_array()[0,:,:,:]
show_3D_array(acquired_data_with_attn_array);

In [ ]:
#%% Plot some profiles
slice = 40
plt.figure()
profile_no_attn = acquired_data_no_attn_array[5,slice,:]
profile_with_attn = acquired_data_with_attn_array[5,slice,:]
profile_attn_factors = attn_factors.as_array()[0,5,slice,:]

plt.plot(profile_no_attn,label='no atten')
plt.plot(profile_with_attn,label='with atten')
plt.plot(profile_no_attn * profile_attn_factors,'bo',label='check')
plt.legend();

# Further things to try

## Back project the data with and without attenuation and compare (easy)

As backprojection multiplies with the transpose matrix, does it "undo" attenuation?

## Model Poisson noise in the data (medium)

This is not so easy unfortunately. Adding noise is done in the [ML_reconstruction](ML_reconstruction.ipynb) exercise if you're stuck.

One important thing to think about when using Poisson-distributed counts is that the mean value is important. This is because the variance is equal to the mean, resulting in lower relative noise for higher counts.

You will therefore have to think about the scale of the image (or the scale of the acquisition data) before getting a Poisson noise sample.

Hint: use `acquired_data.clone()` to create a copy, `numpy.random.poisson`, and `acquired_data.fill()`.

A question that might come to your mind is how you would model a scanner that is less sensitive. This can be done by including another `AcquisitionSensitivityModel` (i.e. another multiplicative factor). We will see this in action for the measured data. However, here you could already just multiply the attenuation data factors with a number before incorporating it into the acquisition sensitivity model, like so

In [ ]:
asm = pet.AcquisitionSensitivityModel(attn_factors* 0.1)

As SIRF (and STIR) reconstructed images are currently proportional to "total counts per voxel", you can use this strategy to incorporate duration and radioactive decay.

## Add an additive background to the model (easy)
This is used to model "accidental coincidences" and "scatter". There are some choices to be made there as the question is if you have a term that should be added before or after attenuation. SIRF allows you to do both.

Use an `AcquisitionData` object that is "constant" (i.e. has the same value everywhere) as model for your background (this is fairly realistic for accidential coincidences).

Hint: read the help for `AcquisitionModel`. Create a simple background by using `AcquisitionData.get_uniform_copy`.

In [ ]:
help(pet.AcquisitionModel)

In [ ]:
help(pet.AcquisitionData.get_uniform_copy)

Check if it modifies the forward projection (it should!), but what about the back-projection?

As you have seen already with the backprojection with ray-tracing, but also when incorporating attenuation, backprojection in PET does **not** give you the inverse. In fact, even proportionality factors will be wrong.

### Mathematical excursion
In many modalities, backprojection is equivalent to the adjoint of the forward operation. However, the adjoint is only defined for linear models. If there is an additive background, the PET forward operator is no longer linear (but it is affine).

The SIRF `backward` operation works for affine models as well. It multiplies data with the Jacobian of the forward model. This is useful for applying the chain rule. When expression images and data as (column) vectors, some mathematical manipulations will show that
$$\frac{\partial}{\partial x} f(A x + b) = A^t \frac{\partial}{\partial y}f(y)|_{y=Ax+b}$$

Note that there is no background term added after multiplication with $A^T$.

Many generic optimisation algorithms, including many implemented in CIL, work with linear operators and their adjoints. SIRF therefore allows you "extract" the linear operator (i.e. $A$ in the above formula) and the background term (i.e. $b$ in the above).

Exercise: use SIRF methods to obtain the linear acquisition model and background, and write the "full" forward and backward operations in terms of those.

Hint: read the help for the `AcquisitionModel`. In particular on the methods `get_constant_term` and `get_linear_acquisition_model`.

In [ ]:
help(pet.AcquisitionModel)

In [ ]:
help(pet.AcquisitionModel.get_linear_acquisition_model)

Note that for linear operators, `backward` is identical to `adjoint`. CIL currently only used the `adjoint` method.

Final note: CIL uses the nomenclature `direct` for `forward`, hence these 2 methods are aliased in SIRF.

## Use other settings for the acquisition data (medium)
In the example above, we started from an existing "template" file. In SIRF, you can create a template based on a number of predefined scanners. An example is given in the [SIRF acquisition_data_from_scanner_info example](https://github.com/SyneRBI/SIRF/blob/v3.0.0/examples/Python/PET/acquisition_data_from_scanner_info.py).

As stated before, SIRF has a limitation on the axial voxel size. Because of this you will have to create images with the correct axial spacing. The [Introductory/acquisition_model_mr_pet_ct notebook](../Introductory/acquisition_model_mr_pet_ct.ipynb) shows you how to construct an image appropriate for your scanner. An alternative is to use the SIRF resampling functions, please check the [Registration notebooks](../Reg).

**Exercise** (hard):

Modify this exercise to use a Siemens mMR template based on the example quoted above.

A difficulty currently is to understand the sirf.STIR coordinate system used by the geometrical shapes. Briefly, shapes are specified in a "gantry" system with coordinates `(a,v,h)` with `a` running in axial direction, `v` in vertical and `h` in horizontal. All are in mm, and `v=0,h=0` is on the scanner axis (but sadly, `a=0` is not in the centre. This is work in progress! (More detail is in the [STIR developers guide](http://stir.sourceforge.net/documentation/STIR-developers-overview.pdf).)

You might not know some PET nomenclature (such as "span" and "view mashing"). Briefly, higher "span" increases the amount of "averaging" used between ring differences, while "view mashing" allows you to add several views. They are used to reduce data size and hence speed up computation. "full" resolution sets `span=1, view_mashing=1`.

This nomenclature is explained in the [STIR glossary](http://stir.sourceforge.net/documentation/STIR-glossary.pdf).

# Final note: modelling of other effects in SIRF?
PET acquisition modelling should also include effects of different detection efficiencies, accidental coincidences and scatter.

This is currently not feasible to do in the simple simulation context given here. For measured data, SIRF can read many scanners' efficiency (often called "normalisation") files, compute an estimate for the accidential coincidences as well as scatter. This is shown in other notebooks.

In practice, many people therefore these terms from their measured data and then modify the image for simple simulations.

SIRF *can* simulate scatter at lower resolution (or at high resolution but at high computational cost). You can have a look at the [SIRF scatter_simulation example](https://github.com/SyneRBI/SIRF/blob/v3.0.0/examples/Python/PET/scatter_simulation.py). (STIR can upsample the low resolution scatter estimate, but this functionality has not yet been exposed in SIRF 3.1).